<a href="https://colab.research.google.com/github/Anubhav2017/Genomics_notebooks/blob/main/inference_kmer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv1D, MaxPooling1D,Reshape, LSTM, Dropout, TimeDistributed, Input
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

from sklearn.utils import shuffle

bases={'A':np.array([0,0,0,1]), 'C':np.array([0,0,1,0]), 'G':np.array([0,1,0,0]), 'T':np.array([1,0,0,0])}

def Kmers_funct(seq, size):
    return [seq[x:x+size].lower() for x in range(len(seq) - size + 1)]

def one_hot_encode_2(y,num_classes):
    y_encoded=[]
    for value in y:
	    letter = [0 for _ in range(num_classes)]
	    letter[value] = 1
	    y_encoded.append(letter)
    return np.array(y_encoded,dtype=np.float16)



In [1]:
import random

def selectref(el,pa,pc,pg,pt):

  dicref={'U':['T'],
  'R':['A','G'],
  'Y':['C','T'],
  'S':['G','C'],
  'W':['A','T'],
  'K':['G','T'],
  'M':['A','C'],
  'B':['C','G','T'],
  'D':['A','G','T'],
  'H':['A','C','T'],
  'V':['A','C','G'],
  'N':['A','T','G','C'],
  }

  dicprobs={'U':[1],
  'R':[pa/(pa+pg),pg/(pa+pg)],
  'Y':[pc/(pc+pt),pt/(pc+pt)],
  'S':[pg/(pc+pg),pc/(pc+pg)],
  'W':[pa/(pa+pt),pt/(pa+pt)],
  'K':[pg/(pt+pg),pt/(pt+pg)],
  'M':[pa/(pa+pc),pc/(pa+pc)],
  'B':[pc/(pc+pg+pt),pg/(pc+pg+pt),pt/(pc+pg+pt)],
  'D':[pa/(pa+pg+pt),pg/(pa+pg+pt),pt/(pa+pg+pt)],
  'H':[pa/(pc+pa+pt),pc/(pc+pa+pt),pt/(pc+pa+pt)],
  'V':[pa/(pc+pg+pa),pc/(pc+pg+pa),pg/(pc+pg+pa)],
  'N':[pa/(pc+pg+pa+pt),pt/(pc+pg+pa+pt),pg/(pc+pg+pa+pt),pc/(pc+pg+pa+pt)]}

  # print(dicprobs[el])
  
  return np.random.choice(dicref[el],p=dicprobs[el])
  # return np.random.choice(dicref[el],p=dicprobs[el])





In [ ]:
import pickle

f=open("unique_elems.pkl","rb")
unique_elems=pickle.load(f)

In [ ]:
id_to_genus={}

for el in unique_elems.keys():
  id_to_genus[unique_elems[el]]=el

In [ ]:
df=pd.DataFrame(columns=["ID","Actual_Genus","Predicted_Genus"])


In [ ]:
from google.colab import drive
drive.mount('/content/drive')
!pip install BioPython

Mounted at /content/drive
     |████████████████████████████████| 2.3 MB 28.9 MB/s 


In [ ]:
from collections import defaultdict
import time
from Bio import SeqIO
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

all_sequences=[]

for sequence in SeqIO.parse('/content/drive/My Drive/ncbi_16s_18s_merged.fasta', "fasta"):
    # unique_elems[sequence.description.split()[1]]=0
    seq=""

    id=sequence.id

    #Use if genus name present in fasta file
    # dftemp={"ID":id,"Actual_Genus":str(sequence.description.split()[1]),"Predicted_Genus":""}

    #Use if genus name not present in fasta file
    dftemp={"ID":id,"Predicted_Genus":""}

    df.append(dftemp,ignore_index=True)

    base_count=dict()
    base_count['A']=0
    base_count['C']=0
    base_count['G']=0
    base_count['T']=0
    base_count['others']=0

    for el in sequence.seq:
      if el not in base_count.keys():

        base_count['others']+=1
      else:
        base_count[el]+=1
    
    na=base_count['A']
    nc=base_count['C']
    ng=base_count['G']
    nt=base_count['T']

    pa=float(na/(na+nc+ng+nt))
    pc=float(nc/(na+nc+ng+nt))
    pg=float(ng/(na+nc+ng+nt))
    pt=float(nt/(na+nc+ng+nt))

    
    for el in sequence.seq:
      if el not in base_count.keys():

        seq += (selectref(el,pa,pc,pg,pt))
      else:
        seq+=el
    
    words = Kmers_funct(seq, size=6)
    joined_sentence = ' '.join(words)
    all_sequences.append(joined_sentence)

X=cv.fit_transform(all_sequences).toarray()

print(X.shape)
    

(24855, 4096)


In [ ]:
i=0
x_data=[]

i=0
for sequence in SeqIO.parse('/content/drive/My Drive/ncbi_16s_18s_merged.fasta', "fasta"):
    x_data.append(X[i])
    i+=1

x_data=np.array(x_data,dtype=np.float32)
x_data=np.reshape(x_data,(-1,4096,1))

In [ ]:

model=Sequential()
model.add((Conv1D(input_shape=(4096,1),filters=6, kernel_size=6, activation='relu')))
model.add((Conv1D(filters=3, kernel_size=6, activation='relu')))
model.add(Flatten())
model.add(Dense(units=1024,activation='relu'))
model.add(Dense(units=4283, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.load_weights("/content/drive/My Drive/training_genomics_kmer/010/cp-0010")
early_stopping = EarlyStopping(monitor='loss', patience=2, mode='min')
model.build()
model.summary()
# history = model.fit(x_train, y_train, epochs=10, batch_size=64, validation_data= (x_test,y_test),callbacks=[cp_callback])


Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 4091, 6)           42        
                                                                 
 conv1d_13 (Conv1D)          (None, 4086, 3)           111       
                                                                 
 flatten_5 (Flatten)         (None, 12258)             0         
                                                                 
 dense_9 (Dense)             (None, 1024)              12553216  
                                                                 
 dense_10 (Dense)            (None, 4283)              4390075   
                                                                 
Total params: 16,943,444
Trainable params: 16,943,444
Non-trainable params: 0
_________________________________________________________________


In [ ]:
from sklearn import metrics

preds=model.predict(x_data)

y_pred_labels=[np.argmax(el) for el in preds]

i=0
for index, row in df.iterrows():
  row["Predicted_Genus"]=id_to_genus[y_pred_labels[i]]
  i+=1


df.to_csv('all_predictions.csv')

In [ ]:
df_misclassified=pd.DataFrame(columns=["ID","Actual_Genus","Predicted_Genus"])

i=0
for index, row in df.iterrows():
  if row["Predicted_Genus"] != row["Actual_Genus"]:
    df_misclassified.append(row)

df_misclassified.to_csv('misclassified.csv')

In [ ]:
# from sklearn.metrics import roc_curve,auc
# from sklearn.metrics import roc_auc_score

# fpr = {}
# tpr = {}
# thresh ={}
# roc_auc={}
# n_class = lm
# # y_true=np.array(y_true)
# preds_training_data=model.predict(x_train)
# # print(preds[0:3])
# # print(y_test[0:3])

# for i in range(n_class):
#   # print(y_train[:,i])

#   if len(np.unique(y_test[:,i]))==2:
#     fpr[i], tpr[i], thresh[i] = roc_curve(y_test[:,i], preds[:,i])
#     roc_auc[i] = auc(fpr[i], tpr[i])